### 0. Create .env file with a token generated on HuggingFace:(https://huggingface.co/settings/tokens)

HUGGINGFACEHUB_API_TOKEN=hf_cccccxxxx

### 1. Langchain - HuggingFace (repo_id="google/flan-t5-large") - Langchain Prompt Template

In [ ]:
%pip install langchain langchain-huggingface

In [2]:
from tqdm import tqdm
from langchain import HuggingFaceHub

model_name = "google/flan-t5-large"
model_kwargs=({"temperature":2,
              "max_length": 640})

llm = HuggingFaceHub(repo_id=model_name, model_kwargs=model_kwargs)

In [4]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

template = "You are a naming consultant for new companies."\
"What is a good name for a {company} that make {product}?"
prompt = PromptTemplate.from_template(template)

chain = LLMChain(llm=llm, prompt=prompt)

rs = chain.invoke({
  "company":"blue sky",
  "product":"colorful pants",
  })
print (rs)
print (rs['text'])


{'company': 'blue sky', 'product': 'colorful pants', 'text': 'sailor'}
sailor


### 2. Simple Sequential Chains

In [16]:
from langchain.chains import SimpleSequentialChain,LLMChain
from langchain import HuggingFaceHub
from langchain import PromptTemplate

model_name = "google/flan-t5-large"
model_kwargs=({"temperature":0,
              "max_length": 100})
llm = HuggingFaceHub(repo_id=model_name, model_kwargs=model_kwargs)

In [17]:
template1 = "What is a good name for a company that makes {product}?"
prompt1 = PromptTemplate.from_template(template1)
chain1 = LLMChain(llm=llm, prompt=prompt1)

template2 = "Write a catch phrase for the company:{company}."
prompt2 = PromptTemplate.from_template(template2)
chain2 = LLMChain(llm=llm, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2],verbose=True)  
catch_phrase = overall_chain.invoke({"Veggie Food"})
print(catch_phrase)



> Entering new SimpleSequentialChain chain...
vegflora
vegflora is a company that makes plants

> Finished chain.
{'input': {'Veggie Food'}, 'output': 'vegflora is a company that makes plants'}


### 3. Langchain Action Agents

In [ ]:
%pip install wikipedia numexpr

In [9]:
from langchain import HuggingFaceHub

model_name = "google/flan-t5-large"
model_kwargs=({"temperature":0,
              "max_length": 100})
llm = HuggingFaceHub(repo_id=model_name, model_kwargs=model_kwargs)

In [ ]:


chain = LLMChain(llm=llm, prompt=prompt)

rs = chain.invoke({
  "company":"blue sky",
  "product":"colorful pants",
  })
print (rs)
print (rs['text'])


In [ ]:
from langchain.agents import get_all_tool_names
from langchain.agents import load_tools, AgentExecutor,create_react_agent,initialize_agent
import pprint
from langchain import PromptTemplate

template = "You are a helpful assisant. Answer question: {question}"
prompt = PromptTemplate.from_template(template)
user_prompt = "Who is Pele?"

# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(get_all_tool_names())
# print(get_all_tool_names())



tools = load_tools(["wikipedia"], llm=llm)
# agent = initialize_agent(tools=tools,llm=llm,agent='zero-shot-react-description', verbose=True)
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
rs =   agent_executor.invoke({"input": user_prompt})
# pprint.pprint(rs)


### 4. Loading Documents and RAG

In [ ]:
%pip install chromadb tiktoken PyMuPDF

In [50]:
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [51]:
# load document
loader = DirectoryLoader(path="./_data/",
                         glob="*.pdf",
                         loader_cls=PyMuPDFLoader
                         )
document = loader.load()

In [30]:
#split into small chunks
# \n\n, \n
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(document)

In [32]:
print(len(texts))
print(texts[7])

9
page_content="the App Store amassing thousands of downloads. Additionally, he's competed and won\nin\xa0several hackathons around the world including PennApps and NWHacks." metadata={'source': '_data/instructor.pdf', 'file_path': '_data/instructor.pdf', 'page': 1, 'total_pages': 3, 'format': 'PDF 1.4', 'title': 'Course: LangChain 101 for Beginners (OpenAI / ChatGPT / LLMOps) | Udemy Business', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m126', 'creationDate': "D:20240716065224+00'00'", 'modDate': "D:20240716065224+00'00'", 'trapped': ''}


In [33]:
from langchain import HuggingFaceHub

model_name = "google/flan-t5-large"
model_kwargs=({"temperature":0,
              "max_length": 100})
llm = HuggingFaceHub(repo_id=model_name, model_kwargs=model_kwargs)

In [34]:
#Chroma MEMORY
#embeddings texts
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
embeddings = HuggingFaceEmbeddings()
store = Chroma.from_documents(texts,embeddings, collection_name="langchain101")

chain = RetrievalQA.from_chain_type(llm, retriever=store.as_retriever())

In [45]:
### Persisted ChromaDB to disk

from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

#instantiate the Chroma object from langchain, using Hugging embedding
#DB name = chromadb_langchain101
persists_directory = "./db/chromadb_langchain101" 
embeddings = HuggingFaceEmbeddings()

# Save to disk
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    collection_name="langchain101",
    persist_directory=persists_directory
)

#query from Chroma object
chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())
# retriever = vectordb.as_retriever()

In [47]:
question = "what does LLM stand for?"
response = chain.invoke(question)
print(response['result'])

Large Language Models


In [49]:
print(chain.invoke("What is instructor name? and how many students did learn from him?"))

{'query': 'What is instructor name? and how many students did learn from him?', 'result': 'Avinash Jain Taught 1 million students how to code'}


In [40]:
question = "what age did he learn programming?"
print(chain.invoke(question))

{'query': 'what age did he learn programming?', 'result': '10'}


In [41]:
question = "what does he hope?"
print(chain.invoke(question))

{'query': 'what does he hope?', 'result': 'to equip you with everything you need to embark on your LangChain adventure'}


In [42]:
question = "what audience of this course?"
print(chain.invoke(question))

{'query': 'what audience of this course?', 'result': 'Python Developers of any skill level interested in building LLM-powered Python applications with LangChain'}


In [48]:
question = "Tell me the requirement of this course?"
response = chain.invoke(question)
print(response['result'])

Basic programming experience with Python
